In [3]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [4]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

2646016/2638744 [==============================] - 2s 1us/step


In [9]:
path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"

In [10]:
path_to_file

'C:\\Users\\Vojta\\.keras\\datasets/spa-eng/spa.txt'

In [11]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [12]:
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [13]:
en_sentence = u"May I borrow this book?"
sp_sentence = u"¿Puedo tomar prestado este libro?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


In [29]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, SPANISH]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in line.split('\t')]
                for line in lines[:num_examples]]
    

    
    return zip(*word_pairs)

In [30]:
num_examples = None

In [31]:
en, sp = create_dataset(path_to_file, num_examples)
print(en[-1])
print(sp[-1])

<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>
<start> si quieres sonar como un hablante nativo , debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un musico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado . <end>


In [45]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)
    print(tensor)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    return tensor, lang_tokenizer

In [46]:
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [47]:
# Try experimenting with the size of that dataset
num_examples = 30000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

[[1, 135, 3, 2], [1, 293, 3, 2], [1, 595, 3, 2], [1, 1428, 3, 2], [1, 766, 3, 2], [1, 848, 27, 2], [1, 4918, 3, 2], [1, 6, 46, 5, 2], [1, 806, 27, 2], [1, 3407, 27, 2], [1, 3408, 27, 2], [1, 128, 27, 2], [1, 4919, 27, 3409, 27, 2], [1, 3409, 27, 2], [1, 3410, 27, 2], [1, 2635, 3, 2], [1, 3411, 27, 2], [1, 71, 27, 2], [1, 1607, 27, 2], [1, 201, 27, 2], [1, 1429, 3, 2], [1, 897, 3, 2], [1, 2147, 3, 2], [1, 766, 3, 2], [1, 1430, 3, 2], [1, 3412, 3, 2], [1, 18, 721, 3, 2], [1, 64, 898, 27, 2], [1, 1431, 32, 8, 27, 2], [1, 3413, 44, 3414, 3, 2], [1, 2636, 3, 2], [1, 70, 3415, 27, 2], [1, 4920, 27, 2], [1, 807, 3, 2], [1, 135, 58, 182, 3, 2], [1, 18, 37, 27, 2], [1, 6, 18, 4921, 5, 2], [1, 6, 2637, 5, 2], [1, 9, 899, 3, 2], [1, 2638, 687, 3, 2], [1, 3416, 3, 2], [1, 12, 2639, 3, 2], [1, 26, 18, 17, 3, 2], [1, 1608, 3, 2], [1, 1432, 3, 2], [1, 281, 3, 2], [1, 3417, 3, 2], [1, 2148, 3, 2], [1, 24, 516, 3, 2], [1, 37, 3418, 3, 2], [1, 24, 1609, 3, 2], [1, 456, 3, 2], [1, 989, 3, 2], [1, 2640, 3

[[1, 36, 3, 2], [1, 36, 3, 2], [1, 36, 3, 2], [1, 36, 3, 2], [1, 679, 3, 2], [1, 313, 37, 2], [1, 313, 3, 2], [1, 60, 7, 2], [1, 435, 37, 2], [1, 435, 37, 2], [1, 435, 37, 2], [1, 73, 37, 2], [1, 73, 37, 2], [1, 73, 37, 2], [1, 844, 37, 2], [1, 844, 3, 2], [1, 86, 37, 2], [1, 86, 37, 2], [1, 86, 37, 2], [1, 132, 37, 2], [1, 132, 3, 2], [1, 36, 44, 3, 2], [1, 36, 44, 3, 2], [1, 1485, 37, 2], [1, 4, 236, 3, 2], [1, 4, 236, 3, 2], [1, 4, 125, 3, 2], [1, 4, 100, 37, 2], [1, 1095, 66, 37, 2], [1, 775, 3, 2], [1, 753, 3, 2], [1, 1300, 37, 2], [1, 1300, 37, 2], [1, 64, 50, 3, 2], [1, 36, 58, 3, 2], [1, 76, 10, 37, 2], [1, 76, 10, 7, 2], [1, 76, 10, 7, 2], [1, 14, 236, 3, 2], [1, 3194, 33, 3, 2], [1, 776, 17, 3, 2], [1, 4, 472, 3, 2], [1, 4, 43, 3, 2], [1, 4, 151, 3, 2], [1, 4, 548, 3, 2], [1, 4, 105, 3, 2], [1, 4, 420, 3, 2], [1, 4, 420, 3, 2], [1, 4, 93, 3, 2], [1, 4, 18, 3, 2], [1, 4, 18, 50, 3, 2], [1, 279, 3, 2], [1, 279, 3, 2], [1, 279, 3, 2], [1, 66, 196, 37, 2], [1, 66, 196, 37, 2], [1

In [48]:
max_length_targ

11

In [49]:
max_length_inp

16

In [50]:
target_tensor.shape

(30000, 11)

In [51]:
split_size = 0.2

In [52]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=split_size)


In [55]:
def convert(lang, tensor):
    translated_tensor = []
    for t in tensor:
        if t != 0:
            word = lang.index_word[t]
            translated_tensor.append(word)
            print(f'{t} ----> {word}')
    return " ".join(translated_tensor)

In [57]:
print("Input Language; index to word mapping")
a = convert(inp_lang, input_tensor_train[0])
print()
print("Target Language; index to word mapping")
b = convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
8 ----> no
324 ----> pueden
7871 ----> pararnos
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
28 ----> they
511 ----> cannot
86 ----> stop
81 ----> us
3 ----> .
2 ----> <end>


In [58]:
print(a)
print(b)

<start> no pueden pararnos . <end>
<start> they cannot stop us . <end>


In [59]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)


In [60]:
print(dataset)

<BatchDataset shapes: ((64, 16), (64, 11)), types: (tf.int32, tf.int32)>


In [61]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 16]), TensorShape([64, 11]))

In [62]:
print(example_input_batch)

tf.Tensor(
[[   1    7   84 ...    0    0    0]
 [   1    4 2337 ...    0    0    0]
 [   1    4   42 ...    0    0    0]
 ...
 [   1   24 1253 ...    0    0    0]
 [   1 5837    3 ...    0    0    0]
 [   1   54   12 ...    0    0    0]], shape=(64, 16), dtype=int32)


In [64]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [65]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print('Encoder output shape: (batch size, sequence length, units)', sample_output.shape)
print('Encoder Hidden state shape: (batch size, units)', sample_hidden.shape)

Encoder output shape: (batch size, sequence length, units) (64, 16, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [67]:
 class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [68]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units)", attention_result.shape)
print("Attention weights shape: (batch_size, sequence_length, 1)", attention_weights.shape)

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 16, 1)


In [69]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state, attention_weights

In [70]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print('Decoder output shape: (batch_size, vocab size)', sample_decoder_output.shape)

Decoder output shape: (batch_size, vocab size) (64, 4935)


In [72]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction='none')


def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [73]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)